https://nbviewer.jupyter.org/urls/bitbucket.org/!api/2.0/snippets/datalytx/5exn54/a620bc20b206369f4a3c74485aac96c0cd16bd3e/files/Google%20Drive%20API%20Tutorial%20(1).json

In [ ]:
# Сначала устанавливаем клиентскую библиотеку Google API для Python
!pip install --upgrade google-api-python-client

In [2]:
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload
from googleapiclient.discovery import build
import pprint
import io

pp = pprint.PrettyPrinter(indent=4)

Указываем Scopes. Scopes - это перечень возможностей,
которыми будет обладать сервис, созданный в скрипте.
Ниже приведены Scopes, которые относятся к API Google Drive
(из официальной документации https://developers.google.com/identity/protocols/googlescopes).
01_drive_scopes.png Как видно, разные Scope предоставляют разный уровень доступа к данным.
Нас интересует Scope "https://www.googleapis.com/auth/drive", который позволяет просматривать,
редактировать, удалять или создавать файлы на Google Диске.

Также указываем в переменной SERVICE_ACCOUNT_FILE путь к файлу с ключами сервисного аккаунта.

In [3]:
SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = '/Users/alexeizh/PycharmProjects/google api/charts-for-um-e7580d8049ff.json'


Создаем Credentials (учетные данные), указав путь к сервисному аккаунту, а также заданные Scopes.
А затем создаем сервис, который будет использовать 3ю версию REST API Google Drive,
отправляя запросы из-под учетных данных credentials.

In [4]:
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('drive', 'v3', credentials=credentials)

Теперь можно получить список файлов и папок, к которым имеет доступ сервис.
Для этого выполним запрос list, выдающий список файлов, со следующими параметрами:

- pageSize - количество результатов выдачи. Можете смело ставить максимальное значение 1000.
У меня стоит 10 результатов, чтобы показать как быть, когда нужно получить результаты по следующей страницы результатов
- параметр files() в fields - параметр, указывающий, что нужно возвращать список файлов,
где в скобках указан список полей для файлов, которые нужно показывать в результатах выдачи.
Со всеми возможными полями можно познакомиться в документации
(https://developers.google.com/drive/api/v3/reference/files) в разделе "Valid fields for files.list".
У меня указаны поля для файлов: id (идентификатор файла в Drive), name (имя) и mimeType (тип файла).
Чуть дальше мы рассмотрим пример запроса с большим количеством полей
- nextPageToken в fields - это токен следующей страницы, если все результаты не помещаются в один ответ

In [5]:
results = service.files().list(pageSize=10,
                               fields="nextPageToken, files(id, name, mimeType)").execute()

In [6]:
pp.pprint(results)

{   'files': [   {   'id': '1pJAkpMmh9lW47XRAJ0opAlb9RGc_p0we',
                     'mimeType': 'application/vnd.google-apps.folder',
                     'name': 'charts_for_UM'},
                 {   'id': '1X3LNVaMBeuA4fHNHGbiO8jOSd1wlh4Z5',
                     'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
                     'name': 'charts.xlsx'}]}


In [7]:
print(len(results.get('files')))

2


Дальше давайте рассмотрим какие ещё поля можно использовать для списка возвращаемых файлов.
Как я уже писал выше, со всеми полями можно ознакомиться по ссылке
https://developers.google.com/drive/api/v3/reference/files. Давайте рассмотрим самые полезные из них:

- parents - ID папки, в которой расположен файл/подпапка
- createdTime - дата создания файла/папки
- permissions - перечень прав доступа к файлу
- quotaBytesUsed - сколько места от квоты хранилища занимает файл (в байтах)

In [8]:
results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name, mimeType, parents, createdTime, permissions, quotaBytesUsed)").execute()

In [10]:
pp.pprint(results.get('files')[1])

{   'createdTime': '2021-05-31T15:20:43.820Z',
    'id': '1X3LNVaMBeuA4fHNHGbiO8jOSd1wlh4Z5',
    'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
    'name': 'charts.xlsx',
    'parents': ['1pJAkpMmh9lW47XRAJ0opAlb9RGc_p0we'],
    'permissions': [   {   'deleted': False,
                           'displayName': 'alexeizhuravlev@charts-for-um.iam.gserviceaccount.com',
                           'emailAddress': 'alexeizhuravlev@charts-for-um.iam.gserviceaccount.com',
                           'id': '13902982436117014445',
                           'kind': 'drive#permission',
                           'role': 'writer',
                           'type': 'user'},
                       {   'deleted': False,
                           'displayName': 'Alexei Zhuravlev',
                           'emailAddress': 'alexeizh@gmail.com',
                           'id': '11165025400987331600',
                           'kind': 'drive#permission',
           

### Теперь рассмотрим как скачивать файлы из Google Drive.

Для этого нам понадобится создать запрос request для получения файла.
После этого задаем интерфейс fh для записи в файл с помощью библиотеки io,
указав в filename название файла (таким образом, можно сохранять файлы из Google Drive сразу с другим названием).
Затем создаем экземпляр класса MediaIoBaseDownload,
передав наш интерфейс для записи файла fh и запрос для скачивания файла request.
Следующим шагом скачиваем файл по небольшим кусочкам (чанкам) с помощью метода next_chunk.

Если из предыдущего описания вам мало что понятно,
не запаривайтесь, просто укажите свой file_id и filename, и всё у вас будет в порядке.

In [17]:
file_id = '1X3LNVaMBeuA4fHNHGbiO8jOSd1wlh4Z5'
request = service.files().get_media(fileId=file_id)
filename = 'charts.xlsx'
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print ("Download %d%%." % int(status.progress() * 100))

Download 100%.


Файлы Google Sheets или Google Docs можно конвертировать в другие форматы,
указав параметр mimeType в функции export_media
(обратите внимание, что в предыдущем примере скачивания файла мы использоали другую функцию get_media).
Например, файл Google Sheets можно конвертировать и скачать в виде файла Excel.

In [ ]:
# file_id = '1X3LNVaMBeuA4fHNHGbiO8jOSd1wlh4Z5'
# request = service.files().export_media(fileId=file_id,
#                                              mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
# filename = 'home/charts_for_UM/charts.xlsx'
# fh = io.FileIO(filename, 'wb')
# downloader = MediaIoBaseDownload(fh, request)
# done = False
# while done is False:
#     status, done = downloader.next_chunk()
#     print ("Download %d%%." % int(status.progress() * 100))


Затем скачанный файл можно загнать в датафрейм.
Это достаточно простой способ получить данные из Google Sheet в pandas-dataframe,
но есть и другие способы, например, воспользоваться библиотекой gspread
(https://riptutorial.com/pandas/example/24056/collect-google-spreadsheet-data-into-pandas-dataframe).

In [18]:
import pandas as pd
df = pd.read_excel('charts.xlsx', sheet_name = 'Чарт Yandex')
df.head(5)

,2021-04-29 00:00:00,2021-05-03,2021-05-04,2021-05-05,2021-05-06,2021-05-07,2021-05-08,2021-05-09,2021-05-10,2021-05-11,...,2021-05-22,2021-05-23,2021-05-24,2021-05-25,2021-05-26,2021-05-27,2021-05-28,2021-05-29,2021-05-30,2021-05-31
0,"1, Masked Wolf, Astronaut In The Ocean","1, Masked Wolf, Astronaut In The Ocean","1, Masked Wolf, Astronaut In The Ocean","1, Masked Wolf, Astronaut In The Ocean","1, Masked Wolf, Astronaut In The Ocean","1, Masked Wolf, Astronaut In The Ocean","1, Masked Wolf, Astronaut In The Ocean","1, Masked Wolf, Astronaut In The Ocean","1, Лев Лещенко, День Победы","1, Masked Wolf, Astronaut In The Ocean",...,"1, HammAli & Navai, Птичка","1, HammAli & Navai, Птичка","1, HammAli & Navai, Птичка","1, HammAli & Navai, Птичка","1, HammAli & Navai, Птичка","1, HammAli & Navai, Птичка","1, HammAli & Navai, Птичка","1, HammAli & Navai, Птичка","1, HammAli & Navai, Птичка","1, HammAli & Navai, Птичка"
1,"2, Artik & Asti, Истеричка","2, Artik & Asti, Истеричка","2, Artik & Asti, Истеричка","2, Artik & Asti, Истеричка","2, Artik & Asti, Истеричка","2, Artik & Asti, Истеричка","2, JONY, Камнепад","2, Artik & Asti, Истеричка","2, Академический Ансамбль песни и пляски Росси...","2, Artik & Asti, Истеричка",...,"2, VAVAN, Лилая, АМУР","2, Manizha, RUSSIAN WOMANРоссия на «Евровидени...","2, Måneskin, ZITTI E BUONIИталия. Победитель н...","2, Måneskin, ZITTI E BUONIИталия. Победитель н...","2, Måneskin, ZITTI E BUONIИталия. Победитель н...","2, Måneskin, ZITTI E BUONIИталия. Победитель н...","2, Masked Wolf, Astronaut In The Ocean","2, Masked Wolf, Astronaut In The Ocean","2, Masked Wolf, Astronaut In The Ocean","2, Masked Wolf, Astronaut In The Ocean"
2,"3, Ramil', Сон","3, Султан Лагучев, Горький вкус","3, Султан Лагучев, Горький вкус","3, Ваня Дмитриенко, Венера-Юпитер","3, Ramil', Сон","3, Ramil', Сон","3, Artik & Asti, Истеричка","3, Лев Лещенко, День Победы","3, Тамара Синявская, Катюша","3, Ramil', Сон",...,"3, twenty one pilots, Saturday","3, Masked Wolf, Astronaut In The Ocean","3, Manizha, RUSSIAN WOMANРоссия на «Евровидени...","3, Masked Wolf, Astronaut In The Ocean","3, Masked Wolf, Astronaut In The Ocean","3, Masked Wolf, Astronaut In The Ocean","3, Måneskin, ZITTI E BUONIИталия. Победитель н...","3, DEAD BLONDE, Мальчик на девятке","3, DEAD BLONDE, Мальчик на девятке","3, DEAD BLONDE, Мальчик на девятке"
3,"4, Ваня Дмитриенко, Венера-Юпитер","4, ХАБИБ, Ягода малинка","4, Ваня Дмитриенко, Венера-Юпитер","4, Ramil', Сон","4, Ваня Дмитриенко, Венера-Юпитер","4, Султан Лагучев, Горький вкус","4, Ramil', Сон","4, Султан Лагучев, Горький вкус","4, Николай Назаров, Отдельный показательный ор...","4, Ваня Дмитриенко, Венера-Юпитер",...,"4, NЮ, kavabanga Depo kolibri, Не держи меня","4, Artik & Asti, Истеричка","4, Masked Wolf, Astronaut In The Ocean","4, DEAD BLONDE, Мальчик на девятке","4, DEAD BLONDE, Мальчик на девятке","4, DEAD BLONDE, Мальчик на девятке","4, DEAD BLONDE, Мальчик на девятке","4, Måneskin, ZITTI E BUONIИталия. Победитель н...","4, Султан Лагучев, Горький вкус","4, Måneskin, ZITTI E BUONIИталия. Победитель н..."
4,"5, Султан Лагучев, Горький вкус","5, Ваня Дмитриенко, Венера-Юпитер","5, Ramil', Сон","5, Султан Лагучев, Горький вкус","5, Султан Лагучев, Горький вкус","5, Ваня Дмитриенко, Венера-Юпитер","5, Султан Лагучев, Горький вкус","5, ХАБИБ, Ягода малинка","5, Марк Бернес, Журавлииз фильма «Летят журавли»","5, Султан Лагучев, Горький вкус",...,"5, MORGENSHTERN, SHOW","5, MORGENSHTERN, SHOW","5, DEAD BLONDE, Мальчик на девятке","5, Manizha, RUSSIAN WOMANРоссия на «Евровидени...","5, MORGENSHTERN, SHOW","5, Artik & Asti, Истеричка","5, Artik & Asti, Истеричка","5, Мот, LYRIQ, Не Бруклин","5, Artik & Asti, Истеричка","5, Artik & Asti, Истеричка"


Рассмотрим простой пример загрузки файла в папку.
Во-первых, нужно указать folder_id - id папки (его можно получить в адресной строке браузера, зайдя в папку,
либо получив все файлы и папки методом list).
Также нужно указать название name, с которым файл загрузится на Google Drive.
Это название может быть отличным от исходного названия файла.
Параметры folder_id и name передаем в словарь file_metadata,
в котором задаются метаданные загружаемого файла.
В переменной file_path указываем путь к файлу.
Создаем объект media, в котором будет указание по какому пути находится загружаемый файл,
а также указание, что мы будем использовать возобновляемую загрузку,
что позволит нам загружать большие файлы.
Google рекомендует (https://developers.google.com/drive/api/v3/manage-uploads)
использовать этот тип загрузки для файлов больше 5 мегабайт.
Затем выполняем функцию create, которая позволит загрузить файл на Google Drive.

In [31]:
folder_id = '1pJAkpMmh9lW47XRAJ0opAlb9RGc_p0we'
name = 'charts.xlsx'
file_path = 'charts.xlsx'
file_metadata = {
                'name': name,
                'parents': [folder_id]
            }
media = MediaFileUpload(file_path, resumable=True)
r = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
pp.pprint(r)

{'id': '1_VR_mC2vl7w_J8DbSoWTbagEG_j_EcGy'}


In [42]:
results = service.files().list(pageSize=10,
                               fields="nextPageToken, files(id, name, mimeType)").execute()

In [43]:
results['files']

[{'id': '1pJAkpMmh9lW47XRAJ0opAlb9RGc_p0we',
  'name': 'charts_for_UM',
  'mimeType': 'application/vnd.google-apps.folder'}]

Можно удалить файл, вызвав функцию delete.

In [41]:
service.files().delete(fileId='14tbaMpoRepvnaWXzbilXfylu6GJCZQBD').execute()

''

In [1]:
import google


In [2]:
import googleapiclient